Analisi semplice
====

In questo notebook iniziamo una analisi semplice per capire il funzionamento di PyTorch e TorchText useremo come riferimento il dataset imdb, non ci interessa il risultato finale ma solo capire il funzionamento 

Per prima cosa impostiamo il seed in modo da avere un esperimento riproducibile



In [2]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Preparare i dati
----

Il concetto principale di TorchText è il Field, definisce come il processare il testo.
Nel nostro Dataset abbiamo in ingresso una stringa con il commento da analizzare e l'etichetta che classifica il commento.
Il parametro dell'oggetto Field ci indica come l'oggetto deve essere processato.
Per comodità definiamo due tipi di campo:

 + TEXT che elabora la recensione

 + LABEL che elabora l'etichetta

TEXT ha impostato come proprietà tokenize='spacy', se non viene passato nulla a questa proprietà la stringa verrà spezzata usando gli spazi.

LABEL è impostato come LabelField una sottoclasse di Field usata per gestire le etichetto

per maggiori info [link](https://github.com/pytorch/text/blob/master/torchtext/data/field.py)


In [3]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

Andiamo a scaricare il dataset

In [4]:
from torchtext import datasets
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

Controlliamo quanti dati contiene e la divisione per classe

In [5]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

pos = 0
neg = 0

for row in train_data[:]:
    label = row.label
    if(label == 'pos'):
        pos += 1
    else:
        neg +=1

print(f'Number of training positive: {pos} negative {neg}')


pos = 0
neg = 0

for row in test_data[:]:
    label = row.label
    if(label == 'pos'):
        pos += 1
    else:
        neg +=1
        
print(f'Number of testing positive: {pos} negative {neg}')

Number of training examples: 25000
Number of testing examples: 25000
Number of training positive: 12500 negative 12500
Number of testing positive: 12500 negative 12500


vediamo come è fatto un esempio

In [6]:
print(vars(train_data.examples[0]))

{'text': ['Bromwell', 'High', 'is', 'a', 'cartoon', 'comedy', '.', 'It', 'ran', 'at', 'the', 'same', 'time', 'as', 'some', 'other', 'programs', 'about', 'school', 'life', ',', 'such', 'as', '"', 'Teachers', '"', '.', 'My', '35', 'years', 'in', 'the', 'teaching', 'profession', 'lead', 'me', 'to', 'believe', 'that', 'Bromwell', 'High', "'s", 'satire', 'is', 'much', 'closer', 'to', 'reality', 'than', 'is', '"', 'Teachers', '"', '.', 'The', 'scramble', 'to', 'survive', 'financially', ',', 'the', 'insightful', 'students', 'who', 'can', 'see', 'right', 'through', 'their', 'pathetic', 'teachers', "'", 'pomp', ',', 'the', 'pettiness', 'of', 'the', 'whole', 'situation', ',', 'all', 'remind', 'me', 'of', 'the', 'schools', 'I', 'knew', 'and', 'their', 'students', '.', 'When', 'I', 'saw', 'the', 'episode', 'in', 'which', 'a', 'student', 'repeatedly', 'tried', 'to', 'burn', 'down', 'the', 'school', ',', 'I', 'immediately', 'recalled', '.........', 'at', '..........', 'High', '.', 'A', 'classic', 'l

Se il dataset ha solo train e test possiamo usare la funzione ```.split()``` per dividere il dataset. Di default viene impostato un rapporto 70/30 che puo essere cambiato usando il parametro ```split_ratio```, possiamo anche passare il ```random_state``` per essere sicuri di ottenere sempre lo stesso risultato

In [7]:
import random

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')

Number of training examples: 17500
Number of validation examples: 7500


Dobbiamo creare un vocabolario, dove ogni parola ha un numero corrispondente, indice (index).

Questo perchè quasi tutti i modelli di machine learning operano su numeri.Verrà costruita una serie di vettori _one-hot_ 
uno per ogni parola.

Un vettore _one-hot_ ha tutti gli elementi a 0 tranne una posizione impostata a 1 (index) la dimensionalità è il totale 
del numero di parole univoche del dizionario, comunemente denominata $V$ 

<img src="./images/one-hot.png" />


Ora il numero di parole all'interno del dataset supera 100'000, questo significa che la dimensionalità di $V$ supera quella cifra.
Questo può dare problemi con la memoria della GPU, andiamo dunque a impostare una dimensione massima del dizionario a 25'000 parole.
Cosa facciamo con le parole che verranno tolte dal dizionario? Le andremo a rimpiazzare cone un token apposito *UNK* che sta per unknown.


In [9]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

controlliamo la dimensione del dizionario

In [10]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


perchè la dimensione del dizionario è 25002? Perchè abbiamo due token aggiuntivi ```<unk>``` per gestire le parole scartate e ```<pad>```.
Cosa serve ```<pad>``` ? 

Quando alimentiamo un modello a volte abbiamo la necessità che tutte le frasi abbiano la stessa lunghezza, perciò le frasi più brevi vengono riempite con ```<pad>``` affinchè tutte arrivino alla stessa lunghezza.

<img src="./images/pad.png" />

Andiamo ad analizzare le parole più frequenti all'interno del dataset di train

In [11]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 203562), (',', 192482), ('.', 165200), ('and', 109442), ('a', 109116), ('of', 100702), ('to', 93766), ('is', 76327), ('in', 61254), ('I', 54000), ('it', 53502), ('that', 49185), ('"', 44277), ("'s", 43315), ('this', 42438), ('-', 36691), ('/><br', 35752), ('was', 35033), ('as', 30384), ('with', 29774)]


possiamo accedere al vocabolario direttamente usando i metodi itos (**i**nteger **to** **s**tring)  e stoi (**s**tring **to** **i**nteger)

In [12]:
TEXT.vocab.itos[:10]

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']

In [13]:
TEXT.vocab.stoi['hello']

13097

In [14]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


Il passo finale della preparazione dei dati è creare gli iteratori. Utilizzaremo questi per navigare i dati nel ciclo training/evaluation.
Gli iteratori ritornano un batch di esempi (già convertiti in tensori) ad ogni iterazione. Utilizzeremo un ```BucketIterator```, questo è uno speciale iteratore che ritorna i Batch di esempi di lunghezze simili, minimizzando il numero di padding per ogni iterazione. 

Vogliamo anche piazzare i tensori ritornati sulla GPU se possibile. E' possibile farlo andando ad impostare un ```torch.device``` all'iteratore

In [15]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

Costruzione del modello
----

Costruiamo un modello giocattolo per fare i nostri esperimenti.
Prima di tutto bisogna creare una classe che estende ```nn.Module```. Dentro ```__init__``` si richiama con super il costruttore di ```nn.Module``` e si definiscono i layer del modello.
Iniziamo creando tre layer:

**Embedding Layer** usato per trasformare il nostro vettore one-hot in un vettore denso. Questo livello è "semplicemente" un Layer Fully connected. Come beneficio abbiamo anche che le parole con significato simile sono mappate vicine in questo spazio dimensionale.

**RNN** La rete RNN prende in ingresso il dense vector e la memoria precedente $h_{(t-1)}$ e la usa per calcolare $h_t$

<img src="./images/rnn.png" />


**Linear Layer** Alla fine il linear layer prende lo hidden state finale e lo usa per produrre l'output $f(h_t)$ nella dimensione voluta.

Il metodo ```forward``` viene chiamato per produrre l'output partendo dagli esempi.

Ogni batch, text è un tensore di dimensione **[sentence lenght,batch size]**, questo verrà trasformato in un vettore one_hot.
La trasformazione in one_hot viene fatta al volo all'interno del modulo di embedding per risparmiare spazio.

L'output viene poi passato al layer di embedding che restituisce la versione vettorializzata della frase, embedded è un tensore di dimensione **[sentence lenght,batch size,embeddind dim]**.

embedded poi alimenta la RNN, Pytorch se non viene passato nessun vettore $h_0$ lo imposta in automatico con tutti i valori a 0.

La RNN ritorna due tensori, ```output``` di dimensione **[sentence lenght,batch size,hidden dim]** e ```hidden``` che rappresenta l'ultimo hidden state. Output rappresenta la concatenazione di tutti gli hidden state. 
Lo verifichiamo con la assert, ```squeeze``` serve per rimuovere la dimensione 1.

alla fine passiamo ```hidden``` alla rete neuronale ```fc``` per ottenere il risultato




In [16]:

import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #la dimensione di text è [sent len, batch size]
        embedded = self.embedding(text)
        
        #la dimensione di embedded è [sent len, batch size, emb dim]
        output, hidden = self.rnn(embedded)
        
        #output contiene tutti gli hidden state concatenati [sent len, batch size, hid dim]
        #la dimensione hidden è [1, batch size, hid dim] contiene l'ultimo stato della RNN
        
        hidden_squeeze = hidden.squeeze(0)
        assert torch.equal(output[-1,:,:], hidden_squeeze)
        return self.fc(hidden_squeeze)

andiamo a instanziare la rete neuronale impostando l'ingresso con dimensionalità del dizionario *one-hot*, e un output di dimensione 1 (problema binario) 

In [17]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [18]:
model

RNN(
  (embedding): Embedding(25002, 100)
  (rnn): RNN(100, 256)
  (fc): Linear(in_features=256, out_features=1, bias=True)
)

creiamoci anche una funzione che ci dice il numero di parametri addestrabili

In [19]:

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,592,105 trainable parameters


Train del modello
----



Per prima cosa bisogna scegliere un ottimizzatore, questo è l'algoritmo che va ad ottimizzare i pesi della rete. 
Scegliamo un semplice **SGD stocastic gradiend descent**. Il primo sono i pesi che verranno ottimizzati, il secondo è il 
**leaning rate** che ci da la velocità con cui l'ottimizzatore cerca di avvicinarsi ad una possibile soluzione.    

In [20]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

Definiamo ora una funzione di loss, questa funzione ci dice quanto siamo lontani dalla soluzione. In Pytorch viene spesso chiamata ```criterion``` .

Useremo la funzione *binary cross entropy with logits*.

Il nostro modello restituisce un numero reale (non compreso tra 0 e 1) mentre le nostre etichette sono due numeri interi 0 e 1, dobbiamo restringere il campo dell'uscita utilizzando una funzione *logit* o *sigmoide*

Una volta ristretto il campo dobbiamo calcolare il loss (quanto siamo lontani dalla soluzione) utilizzando la formula dell'entropia incrociata [binary cross entropy](https://machinelearningmastery.com/cross-entropy-for-machine-learning/).

La funzione  ```BCEWithLogitsLoss ``` fa entrambi questi passi

In [21]:
criterion = nn.BCEWithLogitsLoss()

ora se abbiamo abilitato una GPU spostiamo l'ottimizzatore e il criterion con ```.to()```

In [22]:
model = model.to(device)
criterion = criterion.to(device)

Ci creiamo anche una funzione per valutare visivamente se il nostro modello sta lavorando bene (metrica), usiamo la funzione accuratezza.

Attenzione non è detto che questa metrica vada bene per tutti i problemi.

Prima alimentiamo la rete, poi schiacciamo il risultato tra 0 e 1 e poi lo approssimiamo al primo intero, il round fa si che se il numero è maggiore di 0.5 il risultato sarà 1 altrimenti sarà 0.



In [23]:
def binary_accuracy(preds, y):
    """
    Ritorna l'accuratezza per ogni batch ad esempio se abbiamo indovinato 8 esempi su 10 avremo un risultato di 0.8
    """

    #arrotondo il risultato
    rounded_preds = torch.round(torch.sigmoid(preds))
    
    #cambio il tipo in float in quanto altrimento avrei una divisione intera
    correct = (rounded_preds == y).float()  
    acc = correct.sum() / len(correct)
    return acc

Siamo pronti a creare la funzione di train che andrà ad analizzare tutti i record del dataset a batch con dimensione ```BATCH_SIZE```.

Come prima cosa mettiamo il modello in uno stato di train con ```model.train()``` per abilitare il *dropout* e la *batch normalization*, sebbene in questo modello non ci sono è una buona pratica. 

Per ogni batch andiamo ad azzerare il gradiente, ogni parametro del modello ha un attributo ```grad``` che viene calcoltato con ```criterion```, questo attributo non viene ripulito da pytorch in automatico perciò dobbiamo farlo noi a mano.

Andiamo poi ad alimentare il modello con il testo ```batch.text``` (attenzione che questo attributo cambia per ogni dataset).
In automatico viene chiamata la funzione ```forward```.

Come passo successivo andiamo a togliere una dimensione al risultato per allinearlo al campo ```batch.label```.
Andiamo poi a calcolare il loss e l'accuracy del batch che verranno sommati per calcolare la media alla fine delle iterazioni.

I passi più importanti sono ```loss.backward()``` che si occupa del calcolo del gradiente per ogni peso e ```optimizer.step()``` che aggiorna il modello.

Se avete fatto caso abbiamo detto esplicitamente che ```LABEL ``` è un campo float, questo perchè se non lo avessimo fatto perchè TorchText in automatico mappa i campi come ```LongTensors``` invece il criterion vuole un float  

In [25]:
from tqdm import tqdm

def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator):
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

```evaluate``` è simile al train tranne per il fatto che non dobbiamo aggiornare i pesi e non dobbiamo utilizzare il *dropout* e la *batch normalization*.

Non dobbiamo nemmeno calcolare il gradiente e questo lo si fa con ```no_grad()``` questo fa si che si usi meno memoria e il calcolo risulta più veloce.

Il resto della funzione è uguale a train

In [26]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in tqdm(iterator):

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

Ora è giunto il momento di unire tutti i pezzi e vedere il risultato, ad ogni epoch viene fatto un train su tutti i batch e viene calcolato quanto il modello si comporta bene sul dataset di validation.

Inoltre se il modello ottiene il miglior risultato di loss sul dataset di validation ne salvo i pesi.

In [27]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  1%|▌                                                                                 | 2/274 [00:00<00:21, 12.53it/s]

Epoch: 01
	Train Loss: 0.694 | Train Acc: 50.25%
	 Val. Loss: 0.696 |  Val. Acc: 49.92%


  0%|▎                                                                                 | 1/274 [00:00<00:30,  9.03it/s]

Epoch: 02
	Train Loss: 0.693 | Train Acc: 49.86%
	 Val. Loss: 0.696 |  Val. Acc: 50.01%


  1%|▌                                                                                 | 2/274 [00:00<00:21, 12.61it/s]

Epoch: 03
	Train Loss: 0.693 | Train Acc: 50.19%
	 Val. Loss: 0.696 |  Val. Acc: 50.72%


  1%|▌                                                                                 | 2/274 [00:00<00:23, 11.66it/s]

Epoch: 04
	Train Loss: 0.693 | Train Acc: 49.83%
	 Val. Loss: 0.696 |  Val. Acc: 49.89%


100%|████████████████████████████████████████████████████████████████████████████████| 118/118 [00:01<00:00, 78.33it/s]

Epoch: 05
	Train Loss: 0.693 | Train Acc: 50.03%
	 Val. Loss: 0.696 |  Val. Acc: 50.86%


Ora abbiamo visto che il nostro modello non si è comportato molto bene, poco male non era questo lo scopo dell'esercizio. 
Andiamo a vedere come il modello si comporta sul dataset di test.

In [28]:
model.load_state_dict(torch.load('tut1-model.pt'))
test_loss, test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

100%|████████████████████████████████████████████████████████████████████████████████| 391/391 [00:05<00:00, 77.91it/s]

Test Loss: 0.709 | Test Acc: 47.66%


Andiamo anche a provare il modello su una frase a nostro piacimento

In [31]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

Un esempio di recensione negativa

In [32]:
predict_sentiment(model, "This film is terrible")

0.444013386964798

Un esempio di recensione positiva

In [33]:
predict_sentiment(model, "This film is great")

0.48224857449531555